In [ ]:
pip install langchain-community pypdf

In [ ]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "DEEPSEEK_API_KEY" not in os.environ:
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass(
        prompt="Enter your DEEPSEEK API key (required if using DEEPSEEK): "
    )
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
if not os.environ.get("OPENAI_API_BASE"):
  os.environ["OPENAI_API_BASE"] = getpass.getpass("Enter API base for OpenAI: ")

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/AI大模型与Agent赋能智能运维报告.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

In [ ]:
pip install -qU langchain-openai

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="BAAI/bge-m3")

In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

In [ ]:
pip install -qU langchain-core

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
results = vector_store.similarity_search(
    "AI Agent在运维管理中的实际应用涵盖多个领域，场景和案例有哪些"
)

print(results[0])

In [ ]:
results = await vector_store.asimilarity_search("理和治理。这种集中管理确保多个AI Agent协同工作，共同实现业务目标。例如，AIAgent Orchestrator可以协调IT、客户服务和HR部门")

print(results[0])

In [ ]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("AI Agent自动化IT服务管理，如为IT帮助台打开和关闭工单，或监控网络性能以预防性解决问题。例")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

In [ ]:
embedding = embeddings.embed_query("AI Agent在运维管理中的实际应用与案例有哪些")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "AI Agent在运维管理中的实际应用与案例",
        "将AI Agent嵌入传统运维以实现决策支持，需要哪些关键步骤?",
    ],
)

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "AI Agent在运维管理中的实际应用与案例",
        "将AI Agent嵌入传统运维以实现决策支持，需要哪些关键步骤?",
    ],
)